# YZV311E - DATA MINING 2023-FALL TERM PROJECT 
# AI-Based Traffic Congestion Prediction from Historical Traffic Dat

In [1]:
import pandas as pd
import numpy as np

#### Upload Data

In [2]:
file_path = "C:\\Users\\ysrmhmt\\Desktop\\Lectures\\Data Mining\\freshbursa.csv"
df = pd.read_csv(file_path)


In [3]:
df.head(10)

,Taxi ID,Timestamp,Speed (km/h),Distance (m),Linestring
0,320,2019-01-01 00:00:03.260905+03,48,54,"LINESTRING(28.95837 40.23293,28.95791 40.23328)"
1,320,2019-01-01 00:00:07.224554+03,44,48,"LINESTRING(28.95791 40.23328,28.9575 40.23359)"
2,320,2019-01-01 00:00:11.240787+03,36,39,"LINESTRING(28.9575 40.23359,28.95716 40.23384)"
3,320,2019-01-01 00:00:15.22731+03,37,41,"LINESTRING(28.95716 40.23384,28.9568 40.2341)"
4,320,2019-01-01 00:00:19.231051+03,47,52,"LINESTRING(28.9568 40.2341,28.95636 40.23443)"
5,320,2019-01-01 00:00:23.235291+03,49,54,"LINESTRING(28.95636 40.23443,28.9559 40.23478)"
6,320,2019-01-01 00:00:27.265763+03,49,54,"LINESTRING(28.9559 40.23478,28.95544 40.23513)"
7,320,2019-01-01 00:00:31.357734+03,47,52,"LINESTRING(28.95544 40.23513,28.955 40.23546)"
8,320,2019-01-01 00:00:35.391992+03,48,53,"LINESTRING(28.955 40.23546,28.95456 40.23581)"
9,320,2019-01-01 00:00:39.371636+03,46,50,"LINESTRING(28.95456 40.23581,28.95417 40.23616)"


#### Transform Data

##### Reformat the column "Linestring"

In [4]:
#remove unnecessary string
df["Linestring"] = df["Linestring"].str.replace("(", "")
df["Linestring"] = df["Linestring"].str.replace(")", "")
df["Linestring"] = df["Linestring"].str.replace("LINESTRING" , "")

#split coordinates and create array
df["Linestring"] = df["Linestring"].str.split(r',|\s+')

#rename the column
df = df.rename(columns={"Linestring": "Coordinates"})

In [5]:
df.columns

Index(['Taxi ID', 'Timestamp', 'Speed (km/h)', 'Distance (m)', 'Coordinates'], dtype='object')

##### Convert to rows with 4 coordinates

In [6]:
#Create new array to work with rows which has more than 4 coordinates. It will make easier and less computational
indices_of_repetition = []
for i in df.index:
    # df["Coordinates"][i] = [float(element) for element in df["Coordinates"][i]]
    if len(df["Coordinates"][i]) > 4:
        indices_of_repetition.append(i)
            
new_df = df.loc[indices_of_repetition]

#drop the rows from df
df.drop(indices_of_repetition, inplace=True)

#reset the index after dropping rows
df.reset_index(drop=True, inplace=True)

In [7]:
new_df

,Taxi ID,Timestamp,Speed (km/h),Distance (m),Coordinates
10,320,2019-01-01 00:00:43.374386+03,36,12,"[28.95417, 40.23616, 28.95409, 40.23625, 28.95..."
11,320,2019-01-01 00:00:47.387559+03,32,9,"[28.95399, 40.2365, 28.95392, 40.23657, 28.953..."
12,320,2019-01-01 00:00:51.388046+03,39,10,"[28.95363, 40.23666, 28.95351, 40.23667, 28.95..."
18,320,2019-01-01 00:01:15.441994+03,36,30,"[28.95063, 40.23873, 28.95033, 40.23889, 28.95..."
19,320,2019-01-01 00:01:19.445403+03,23,5,"[28.95026, 40.23895, 28.95025, 40.239, 28.9502..."
...,...,...,...,...,...
1048566,881,2019-01-02 10:58:33.518337+03,0,2,"[29.0637, 40.17657, 29.06368, 40.17659, 29.063..."
1048569,881,2019-01-02 11:00:03.559054+03,0,1,"[29.06379, 40.17653, 29.0638, 40.17652, 29.063..."
1048570,881,2019-01-02 11:00:33.551994+03,1,0,"[29.06377, 40.17653, 29.06378, 40.17653, 29.06..."
1048572,881,2019-01-02 11:01:33.542553+03,1,1,"[29.06376, 40.17656, 29.06375, 40.17657, 29.06..."


In [8]:
#Divide the coordinates into 4 and create new rows
new_rows = []

# Iterate over rows in the new_df DataFrame
for _, row in new_df.iterrows():
    len_row = len(row["Coordinates"])
    n = 0
    
    # Iterate over pairs of coordinates with a step of 2
    while n <= len_row - 4:
        x1, x2, x3, x4 = row["Coordinates"][n], row["Coordinates"][n + 1], row["Coordinates"][n + 2], row["Coordinates"][n + 3]

        
        # Create a new row with the extracted coordinates
        new_row = {
            "Taxi ID": row["Taxi ID"],
            "Timestamp": row["Timestamp"],
            "Speed (km/h)": row["Speed (km/h)"],
            "Distance (m)": row["Distance (m)"],
            "Coordinates": [x1, x2, x3, x4]
        }
        
        # Append the new row to the list
        new_rows.append(new_row)
        
        n += 2


In [9]:
# Append the new rows to the df DataFrame
cor4_df = pd.DataFrame(new_rows)
# Concatenate df and new_df
df = pd.concat([df, cor4_df], ignore_index=True)

In [10]:
df.head(10)

,Taxi ID,Timestamp,Speed (km/h),Distance (m),Coordinates
0,320,2019-01-01 00:00:03.260905+03,48,54,"[28.95837, 40.23293, 28.95791, 40.23328]"
1,320,2019-01-01 00:00:07.224554+03,44,48,"[28.95791, 40.23328, 28.9575, 40.23359]"
2,320,2019-01-01 00:00:11.240787+03,36,39,"[28.9575, 40.23359, 28.95716, 40.23384]"
3,320,2019-01-01 00:00:15.22731+03,37,41,"[28.95716, 40.23384, 28.9568, 40.2341]"
4,320,2019-01-01 00:00:19.231051+03,47,52,"[28.9568, 40.2341, 28.95636, 40.23443]"
5,320,2019-01-01 00:00:23.235291+03,49,54,"[28.95636, 40.23443, 28.9559, 40.23478]"
6,320,2019-01-01 00:00:27.265763+03,49,54,"[28.9559, 40.23478, 28.95544, 40.23513]"
7,320,2019-01-01 00:00:31.357734+03,47,52,"[28.95544, 40.23513, 28.955, 40.23546]"
8,320,2019-01-01 00:00:35.391992+03,48,53,"[28.955, 40.23546, 28.95456, 40.23581]"
9,320,2019-01-01 00:00:39.371636+03,46,50,"[28.95456, 40.23581, 28.95417, 40.23616]"


In [11]:
df[['x1', 'y1', 'x2', 'y2']] = pd.DataFrame(df['Coordinates'].to_list(), index=df.index)
df[['x1', 'y1', 'x2', 'y2']] = df[['x1', 'y1', 'x2', 'y2']].astype(float)

In [12]:
new = pd.concat([df["Taxi ID"], df['x1'], df['y1']], axis=1)


In [13]:
new.head()

,Taxi ID,x1,y1
0,320,28.95837,40.23293
1,320,28.95791,40.23328
2,320,28.95750,40.23359
3,320,28.95716,40.23384
4,320,28.95680,40.23410


In [14]:
# df["Longitude"] = (df["x1"] + df["x2"]) / 2
# df["Latitude"] = (df["y1"] + df["y2"]) / 2

df["Longitude"] = df["x1"]
df["Latitude"] = df["y1"]
df = df.drop(['x1', 'y1', 'x2', 'y2', 'Coordinates'], axis=1)

In [15]:
df.to_csv("Clean_Data", index=False)